In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append(os.path.abspath("../../../DNANEXUS/Updated_data"))
import utilsfiles
from simpledbf import Dbf5
from utilsfiles import recode_reads

In [ ]:
gp_data = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/gp_clinical_6_june.csv")

In [ ]:
coding1834 = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/codings/coding1834.tsv", sep="\t")

In [ ]:
coding1834["coding"] = coding1834["coding"].astype(str)

In [ ]:
coding1835 = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/codings/coding1835.tsv", sep="\t")

In [ ]:
coding1835["coding"] = coding1835["coding"].astype(str)

In [ ]:
dict1834 = dict(zip(coding1834["coding"], coding1834["meaning"]))
dict1835 = dict(zip(coding1835["coding"], coding1835["meaning"]))

In [ ]:
gp_data["icd10"] = gp_data["read_2"].map(dict1834)
gp_data["icd10"] = gp_data["read_3"].map(dict1835)


In [ ]:
gp_data_drop = gp_data.dropna(subset="icd10")

In [ ]:
gp_data_drop.to_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/gpmappedicd1018341835.csv")

In [ ]:
gp_data_drop

In [ ]:
dbf3 = Dbf5('/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/codings/readbrowser/Standard/V3/ICD10.DBF')
df3 = dbf3.to_dataframe()
dbf2 = Dbf5('/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/codings/readbrowser/Standard/V2/ICD10.DBF')
df2 = dbf2.to_dataframe()
df3 = recode_reads(df3)
df2 = recode_reads(df2)

In [ ]:
df2 = df2[["READ_CODE","value"]]
df3= df3[["READ_CODE","value"]]
df3_dict = dict(zip(df3["READ_CODE"], df3["value"]))
df2_dict = dict(zip(df2["READ_CODE"], df2["value"]))

In [ ]:
gp_data["icd10"] = gp_data["read_2"].map(df2_dict)
gp_data["icd10"] = gp_data["read_3"].map(df3_dict)

In [ ]:
gp_datadropnhs = gp_data.dropna(subset="icd10")

In [ ]:
gp_datadropnhs.rename(columns={"icd10" : "icd10_analytical"}, inplace=True)

In [ ]:
gp_mapped = pd.merge(gp_datadropnhs , gp_data_drop, on=["eid","data_provider","event_dt","read_2","read_3","value1","value2","value3"])

In [ ]:
gp_mapped_dropna = gp_mapped.dropna(subset=["icd10_analytical", "icd10"] ,how="all")

In [ ]:
gp_mapped_dropna.to_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/gpmappedicd10_analytical.csv")

In [ ]:
gp_mapped_dropna[["eid","event_dt","icd10","icd10_analytical"]].to_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/gpmappedicd10_analytical.csv")

In [ ]:
gp_mapped_dropna[["eid","event_dt","icd10","icd10_analytical"]]